In [41]:
# import libraries
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import (
    TokenTextSplitter)

import pandas as pd
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings

import os
import re

Exception ignored in: Exception ignored in: <generator object tqdm.__iter__ at 0x3100a0820>
Traceback (most recent call last):
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/tqdm/notebook.py", line 157, in display
    pbar.value = self.n
    ^^^^^^^^^^
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/traitlets/traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/traitlets/traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/traitle

In [34]:
# import data
markdown_files = []
for root, dirs, files in os.walk("../data/processed_files"):
    for file in files:
        if file.lower().endswith('.md'):
            markdown_files.append(os.path.join(root, file))
    

In [35]:
# Iterate over the file paths
loaded_documents = []
for doc in markdown_files:
    try:
        loader = UnstructuredMarkdownLoader(doc)
        documents = loader.load()
        loaded_documents.extend(documents)
        print(f"Loaded: {doc}")
    except Exception as e:
        print(f"Error loading {doc}: {str(e)}")

Loaded: ../data/processed_files/Fortportal FY2020/Fortportal Regional Referral Hospital Report of Auditor General 2020.md
Loaded: ../data/processed_files/Fortportal FY2021/Fortportal Regional Referral Hospital Report FY20202021.md
Loaded: ../data/processed_files/Gulu FY2021/Gulu DLG Report of Auditor General 2021.md
Loaded: ../data/processed_files/CAG FY2022/Annual Consolidated OAG audit reports 2022.md
Loaded: ../data/processed_files/MWTS FY2021/MWTS Report of Auditor General 2021.md
Loaded: ../data/processed_files/Fortportal FY2022/Fortportal Regional Referral Hospital Report of Auditor General 2022.md
Loaded: ../data/processed_files/Gulu FY2022/Gulu DLG Report of Auditor General 2022.md
Loaded: ../data/processed_files/CAG FY2021/Annual Consolidated OAG audit reports 2021.md
Loaded: ../data/processed_files/MWTS FY2022/MWTS Report of Auditor General 2022.md


In [36]:
# 2. Custom function to normalize text
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove special characters (customize as needed)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Apply normalization to each document
for doc in loaded_documents:
    doc.page_content = normalize_text(doc.page_content)

In [37]:
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=150, length_function=len
)
docs = text_splitter.split_documents(loaded_documents)

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-70B",
    task="text-generation",
    max_new_tokens=512,
    timeout=5,
    repetition_penalty=1.03,
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

generator_llm = llm
critic_llm = llm
embeddings = embeddings

/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {  # uniform distribution
    simple: 0.1,
    reasoning: 0.35,
    multi_context: 0.2,
    conditional: 0.35
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(docs, 10, distributions, raise_exceptions=False) 
testset.to_pandas()

embedding nodes:   0%|          | 0/3810 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
import time
import random
import logging

def make_request_with_backoff(max_retries=5, max_wait_time=60):
    for attempt in range(max_retries):
        try:
            logging.info(f"Attempt {attempt + 1} of {max_retries}")
            testset = generator.generate_with_langchain_docs(docs, 50, distributions, raise_exceptions=False)
            logging.info(f"Successfully generated {len(testset)} items")
            return testset
        except Exception as e:
            if "429" in str(e):
                wait_time = min((2 ** attempt) + random.uniform(0, 1), max_wait_time)
                logging.warning(f"Rate limit hit. Waiting {wait_time:.2f} seconds.")
                time.sleep(wait_time)
            else:
                logging.error(f"Unexpected error: {str(e)}")
                raise e
    raise Exception("Max retries reached")

# Configure logging
logging.basicConfig(level=logging.INFO)

# Use the function
testset = make_request_with_backoff()

INFO:root:Attempt 1 of 5
ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-3747' coro=<as_completed.<locals>.sema_coro() running at /Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/ragas/executor.py:36> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/asyncio/tasks.py:618]>
/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/ragas/executor.py:109: RuntimeWarning: coroutine 'Executor.wrap_callable_with_index.<locals>.wrapped_callable_async' was never awaited
  def wrap_callable_with_index(self, callable: t.Callable, counter):
Exception ignored in: <coroutine object as_completed.<locals>.sema_coro at 0x31c720220>
Traceback (most recent call last):
  File "/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/ragas/executor.py", line 36, in sema_coro
    async with semaphore:
  File "/Users/jonas/

embedding nodes:   0%|          | 0/3810 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# works but is quite expensive with OpenAI API token. 